In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = "https://www.opindia.com/latest-news/"
data = get(url)

In [3]:
data

<Response [200]>

In [4]:
soup = BeautifulSoup(data.content,"html.parser")

In [5]:
def fetch_article(url):
    data = get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_=["entry-title td-module-title"]):
        articles.append(i.find('a')["title"])
    return articles

In [6]:
urllist = ["https://www.opindia.com/latest-news/"]
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/"+str(i)+"/"
    urllist.append(url)

In [7]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [9]:
p_art =[]
for i in all_articles:
    i = i.upper()
    import re
    i = re.sub("[^A-Z0-9 ]","",i)
    from nltk.stem import PorterStemmer
    tk_q = i.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [11]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [12]:
cl_res.labels_

array([2, 3, 2, ..., 4, 4, 4])

In [13]:
Q = pd.DataFrame(p_art,columns=["Article"])

In [14]:
Q["Cluster"] = cl_res.labels_

In [15]:
Q[Q["Cluster"]==4].head(20)

,Article,Cluster
5,ARE HINDU WAST TIME ON THE HIJAB ROW HERE IS ...,4
10,ANTICAA RIOTER IN UP WILL STILL PAY FOR THE D...,4
12,I GOT 25 CRORE BUT GAVE DETAIL OF 266 CRORE O...,4
14,CENTRAL GOVT CLAMP DOWN ON PROKHALISTAN CHANN...,4
17,AS THE KASHMIR FILE GET READI TO SHOW STORI O...,4
18,DID SHEKHAR GUPTA THEPRINT USE PORN FILM INTR...,4
21,HARVARD STUDENT GET A LESSON ON YOGI ADITYANA...,4
25,CHRONOLOG FACT DONT MATCH UP HERE IS WHI EVEN...,4
26,BURQA ROW IN KARNATAKA THREATEN COMMUN HARMON...,4
32,ARE HINDU WAST TIME ON THE HIJAB ROW HERE IS ...,4


In [16]:
E = {1 : "History",
2 : "Country",
3 : "Politics",
4 : "Social",
0 : "Religion"}

In [17]:
R = []
for i in Q.Cluster:
    R.append(E[i])
    
Q["Category"] = R

In [18]:
Q

,Article,Cluster,Category
0,TAMIL NADU BJP MAKE SIGNIFIC GAIN IN LOCAL BO...,2,Country
1,RSS ORGANIS HINDU SOCIETI TO MAKE COUNTRI STR...,3,Politics
2,AMIDST ONGO UP POLL RUMOUR EMERG SP CHIEF AKH...,2,Country
3,MEET THE EXPERT BEHIND THE UNHRC STATEMENT SU...,0,Religion
4,SWAPNA SURESH MAIN ACCUS IN KERALA GOLD SMUGG...,0,Religion
...,...,...,...
2075,ISI TRI TO SABOTAG PUNJAB ELECT THINK IT IS N...,4,Social
2076,IMRAN KHAN IS AN INTERN BEGGAR JAMAATEISLAMI ...,4,Social
2077,DRAG THAT CAMR BTCH OUT OF THE ROOM WHEN SAMA...,4,Social
2078,TEXA FBI IDENTIFI SYNAGOGU HOSTAGETAK AS BRIT...,4,Social
